# How to build a simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to store memory in a json file.


A chatbot app is a software application designed to simulate conversation with human users. It uses artificial intelligence (AI) to understand what someone is saying and to respond with appropriate answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-chatbot.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-simple-chatbot**.

## Trick to avoid the nasty deprecation warnings from LangChain

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [17]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [18]:
#!pip install python-dotenv

In [19]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
google_api_key = os.environ["GOOGLE_API_KEY"]

from langchain_google_genai import ChatGoogleGenerativeAI

chatbot = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)


#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [20]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [21]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

* Human Message: the user input.

In [22]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

#### Call the ChatModel (the LLM)

In [23]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content='That\'s great! Blue is a very popular and versatile color. It\'s often associated with:\n\n*   **Calmness and serenity:** Think of the ocean or the sky.\n*   **Trust and stability:** Many businesses use blue in their branding.\n*   **Intelligence and wisdom:** It\'s often used in educational settings.\n*   **Sadness:** (Hence the phrase "feeling blue")\n\nDo you have a specific shade of blue that you prefer? Is there a reason why blue is your favorite color? I\'m curious to know more!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--bd20d267-a0fa-416e-8f15-c73af20f1fc6-0', usage_metadata={'input_tokens': 6, 'output_tokens': 119, 'total_tokens': 125, 'input_token_details': {'cache_read': 0}})

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Check if the Chatbot remembers your favorite color.

In [24]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="As a large language model, I have no way of knowing what your favorite color is. I have no access to personal information about you. To figure it out, you'd have to tell me!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--4123a7b8-2c0b-40bf-b73f-1b52a4f17005-0', usage_metadata={'input_tokens': 6, 'output_tokens': 42, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [25]:
#!pip install langchain_community

In [26]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [27]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [28]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [29]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [30]:
chain.invoke("hello!")

{'content': 'hello!',
 'messages': [HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is Julio', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Julio! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Julio.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!', additiona

In [31]:
chain.invoke("my name is Trung")

{'content': 'my name is Trung',
 'messages': [HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is Julio', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Julio! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Julio.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!',

In [32]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is Julio', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Julio! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Julio.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello!',

* Check the file messages.json in the root directory.
* This is just a simple example, in the real world you probably will not save your memory in a json file. 
* And remember: the context window is limited and it affects to the cost of using chatGPT API.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-chatbot.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-chatbot.py